In [ ]:
# -*- coding: utf-8 -*-
'''
TopQuant-简称TQ极宽智能量化回溯分析系统，培训课件-配套教学python课件程序

Top极宽量化(原zw量化)，Python量化第一品牌 
by Top极宽·量化开源团队 2017.10.1 首发

网站： www.TopQuant.vip      www.ziwang.com
QQ群: Top极宽量化1群，124134140
      Top极宽量化2群，650924099
      Top极宽量化3群，450853713
  
'''
#1
import os,arrow
import pandas as pd
import numpy as np
#2
import keras
from keras import initializers,models,layers
from keras.models import Sequential,load_model
from keras.layers import Flatten,Dense, Input, Dropout, Embedding,SimpleRNN,Bidirectional,LSTM,Conv1D, GlobalMaxPooling1D,Activation,MaxPooling1D,GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras.utils import plot_model

#3
import tensorlayer as tl
import tensorflow as tf

#4
import zsys
import ztools as zt
import ztools_str as zstr
import ztools_data as zdat
import ztools_draw as zdr
import ztools_tq as ztq
import zpd_talib as zta
#
import zai_keras as zks

#
#------------------------------------

#1
print('\n#1,set.sys')
pd.set_option('display.width', 450)    
pd.set_option('display.float_format', zt.xfloat3)    
rlog='ailib/log_tmp'
if os.path.exists(rlog):tf.gfile.DeleteRecursively(rlog)

#2.1
print('\n#2.1,读取数据')
rss,fsgn,ksgn='ailib/TDS/','TDS2_zz500','avg'
xlst=zsys.TDS_xlst9
zt.prx('xlst',xlst)
num_in,num_out=len(xlst),1
print('\nnum_in,num_out:',num_in,num_out)

#
df_train,df_test,x_train,y_train,x_test, y_test=zdat.frd_TDS(rss,fsgn,ksgn,xlst)
print('\ndf_test.tail()')
print(df_test.tail())
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))

#
#2.2
print('\n#2.2,转换数据格式shape')
rxn,txn=x_train.shape[0],x_test.shape[0]
x_train,x_test = x_train.reshape(rxn,num_in,-1),x_test.reshape(txn,num_in,-1)
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))



#3
print('\n#3,model建立神经网络模型')
mx=zks.lstm010(num_in,num_out)
#
mx.summary()
plot_model(mx, to_file='tmp/lstm010mx.png')


#4 模型训练
print('\n#4 模型训练 fit')
tbCallBack = keras.callbacks.TensorBoard(log_dir=rlog,write_graph=True, write_images=True)
tn0=arrow.now()
mx.fit(x_train, y_train, epochs=2000, batch_size=512*24,callbacks=[tbCallBack])
tn=zt.timNSec('',tn0,True)
mx.save('tmp/lstm010mx.dat')

#5 利用模型进行预测 predict
print('\n#5 模型预测 predict')
tn0=arrow.now()
y_pred = mx.predict(x_test)
tn=zt.timNSec('',tn0,True)
df_test['y_pred']=zdat.ds4x(y_pred,df_test.index,True)
df_test.to_csv('tmp/df_lstm010mx.csv',index=False)


#6
print('\n#6 acc准确度分析')
print('\nky0=10')
df=df_test
dacc,dfx,a10=ztq.ai_acc_xed2ext(df.y,df.y_pred,ky0=10,fgDebug=True)


Using TensorFlow backend.



#1,set.sys

#2.1,读取数据

 xlst
['open', 'high', 'low', 'close', 'volume', 'avg', 'ma_2', 'ma_3', 'ma_5', 'ma_10', 'ma_15', 'ma_20', 'ma_25', 'ma_30', 'ma_50', 'ma_100', 'xyear', 'xmonth', 'xday', 'xday_week', 'xday_year', 'xweek_year']

num_in,num_out: 22 1

df_test.tail()
       open  high   low  close     volume   avg  ma_2  ma_3  ma_5  ma_10      ...       xyear  xmonth  xday  xday_week  xday_year  xweek_year     y  price  price_next  price_change
31077 4.310 4.350 4.300  4.350  95042.000 4.330 4.340 4.330 4.310  4.330      ...        1970       1     1          3          1           0 4.350  4.330       4.350       100.462
31078 4.340 4.350 4.310  4.330  81712.000 4.330 4.330 4.340 4.320  4.320      ...        1970       1     1          3          1           0 4.350  4.330       4.350       100.462
31079 4.360 4.360 4.320  4.340  87998.000 4.350 4.340 4.340 4.330  4.320      ...        1970       1     1          3          1           0 4.360  4.350       4.360       100.230
310